In [19]:
import numpy as np
import pandas as pd
import scann
from lightfm import LightFM
from scipy.sparse import coo_matrix, save_npz, load_npz

In [4]:
# -----------------=[ Data reading ]=----------------

usersM = pd.read_csv('./data/users.csv')
gamesM = pd.read_csv('./data/games.csv')
recommendationsM = pd.read_csv('./data/recommendations.csv')
# gamesMetadataM = pd.read_json('./data/games_metadata.json')


In [5]:
usersM.head()

,user_id,products,reviews
0,7360263,359,0
1,14020781,156,1
2,8762579,329,4
3,4820647,176,4
4,5167327,98,2


In [6]:
gamesM.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


In [7]:
recommendationsM.head()

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,51580,0
1,304390,4,0,2017-02-17,False,11.5,2586,1
2,1085660,2,0,2019-11-17,True,336.5,253880,2
3,703080,0,0,2022-09-23,True,27.4,259432,3
4,526870,0,0,2021-01-10,True,7.9,23869,4


In [ ]:
# gamesMetadataM.head()

In [8]:
# -------------------=[ Mappers ]=-------------------

usersV = usersM['user_id'].unique()
gamesV = gamesM['app_id'].unique()

uID_to_idx = {userId: idx for idx, userId in enumerate(usersV)}
gID_to_idx = {gameId: idx for idx, gameId in enumerate(gamesV)}
idx_to_uID = {idx: userId for idx, userId in enumerate(usersV)}
idx_to_gID = {idx: gameId for idx, gameId in enumerate(gamesV)}

In [17]:
# ---------------=[ Data merging ]=-------------------

row_indices = []
col_indices = []
data = []

for _,row in recommendationsM.iterrows():
    user_id = row['user_id']
    app_id = row['app_id']
    is_recommended = row['is_recommended']

    row_indices.append(uID_to_idx[user_id])
    col_indices.append(gID_to_idx[app_id])
    data.append(1 if is_recommended else 0)

In [22]:
# -------------=[ User x Game Matrix ]=---------------

USERxGAME = load_npz('./data/rating_matrix_sparse.npz').tocsr()

In [ ]:
# -------------=[ Game feature Matrix ]=-------------
# TODO

In [38]:
def fit(model, userxgame, epochs=10):
  for epoch in range(epochs):
    model.fit_partial(userxgame, epochs=1, num_threads=4)
    itemEmbeddings = model.item_embeddings
    userEmbeddings = model.user_embeddings
    itemBiases = model.item_biases
    userBiases = model.user_biases

    print(f'Epoch {epoch} completed!')
    
    np.save('./data/model/item_embeddings.npy', itemEmbeddings)
    np.save('./data/model/user_embeddings.npy', userEmbeddings)
    np.save('./data/model/item_biases.npy', itemBiases)
    np.save('./data/model/user_biases.npy', userBiases)

def loadModel():
  itemEmbeddings = np.load('./data/model/item_embeddings.npy')
  userEmbeddings = np.load('./data/model/user_embeddings.npy')
  itemBiases = np.load('./data/model/item_biases.npy')
  userBiases = np.load('./data/model/user_biases.npy')

  model = LightFM(learning_schedule='adagrad', loss='warp')

  model.item_embeddings = itemEmbeddings
  model.user_embeddings = userEmbeddings
  model.item_biases = itemBiases
  model.user_biases = userBiases

  return model

In [40]:
# ----------------=[ Model training ]=---------------

model = loadModel()
fit(model, USERxGAME, 3)

Epoch 0 completed!
Epoch 1 completed!
Epoch 2 completed!


In [ ]:
itemEmbeddings = itemEmbeddings / np.linalg.norm(itemEmbeddings, axis=1, keepdims=True)
userEmbeddings = userEmbeddings / np.linalg.norm(userEmbeddings, axis=1, keepdims=True)

scannSearcher = scann.scann_ops_pybind.builder(itemEmbeddings, 5, "dot_product").score_ah(2).build()

In [ ]:
# -----------------=[ Prediction ]=------------------

def predict(userVec, k):
  pass


In [ ]:
# ------------------=[ Metrics ]=--------------------